In [ ]:
from __future__ import print_function

import sys
import numpy as np
import pandas as pd
from tqdm import tqdm, tqdm_gui, tqdm_notebook
from PIL import Image, ImageFile
import os

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [ ]:
import matplotlib.pyplot as plt
from os.path import isfile
import torch.nn.init as init
import torch
import torch.nn as nn
print(os.listdir("../input"))
from sklearn.model_selection import train_test_split, StratifiedKFold
from torch.utils.data import Dataset
from torchvision import transforms
from torch.optim import Adam, SGD, RMSprop
import time
from torch.autograd import Variable
import torch.functional as F
from tqdm import tqdm
from sklearn import metrics
import urllib
import pickle
import torch.nn.functional as F
from torchvision import models
import seaborn as sns
import random

In [ ]:
IS_LOCAL = False
if(IS_LOCAL):
    PATH="../input/APTOS 2019 Blindness Detection/"
else:
    PATH="../input/"
os.listdir(PATH)

In [ ]:
%%time
train_df = pd.read_csv(os.path.join(PATH,'train.csv'), dtype={'id_code': np.unicode_, 'time_to_failure': np.int32})

In [ ]:
print("Train: rows:{} cols:{}".format(train_df.shape[0], train_df.shape[1]))

In [ ]:
pd.options.display.precision = 15
train_df.head(10)

In [ ]:
import cv2

In [ ]:
x_train_shape = []

In [ ]:
for im_id in tqdm_notebook(train_df["id_code"]):
    seg = cv2.imread('../input/train_images/' + im_id + '.png', cv2.IMREAD_UNCHANGED)
    x_train_shape.append(seg.shape)

In [ ]:
min(x_train_shape), max(x_train_shape)

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [ ]:
seed_everything(1234)
TTA         = 5
num_classes = 1

In [ ]:
def crop_image1(img,tol=7):
    # img is image data
    # tol  is tolerance
        
    mask = img>tol
    return img[np.ix_(mask.any(1),mask.any(0))]

def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
    #         print(img1.shape,img2.shape,img3.shape)
            img = np.stack([img1,img2,img3],axis=-1)
    #         print(img.shape)
        return img

In [ ]:
def image(idx):
    label = train_df.diagnosis.values[idx]
    label = np.expand_dims(label, -1)
    image = cv2.imread('../input/train_images/' + train_df.id_code.values[idx] + '.png', cv2.IMREAD_UNCHANGED)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = crop_image_from_gray(image)
    image = cv2.resize(image, (192, 256))
    return image, label

In [ ]:
image(0)

In [ ]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./256,
    rotation_range=30,
    shear_range = 0.1,
    brightness_range=[0.5, 1.5],
    zoom_range = [0.5, 1.5],
    horizontal_flip = True,
    vertical_flip=False,
    validation_split=0.2)

In [ ]:
train_df = pd.read_csv('../input/train.csv')
train_df['diagnosis'] = train_df['diagnosis'].astype('str')
train_df['id_code'] = train_df['id_code'].astype(str)+'.png'

In [ ]:
data_train = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory = '../input/train_images',
    target_size=(192, 256),
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=None,
    save_to_dir=None,
    save_prefix='',
    save_format='png',
    follow_links=False,
    x_col="id_code",
    y_col="diagnosis",
    subset='training',
    interpolation='nearest')

In [ ]:
data_test = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory = '../input/train_images',
    target_size=(192, 256),
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=None,
    save_to_dir=None,
    save_prefix='',
    save_format='png',
    follow_links=False,
    x_col="id_code",
    y_col="diagnosis",
    subset='validation',
    interpolation='nearest')

In [ ]:
input_shape = ()
model = Sequential()
model.add(Conv2D(16, (2, 3), padding='same', activation='relu', input_shape=(192, 256, 3)))
model.add(Conv2D(32, (2, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 3)))
model.add(Dropout(0.15))

model.add(Conv2D(16, (3, 4), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 4)))
model.add(Dropout(0.10))

model.add(Conv2D(32, (2, 3), padding='same', activation='relu'))
model.add(Conv2D(16, (2, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 3)))
model.add(Dropout(0.05))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

In [ ]:
y_train = train_df['diagnosis']
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train)
num_classes = y_train.shape[1]

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.adam(lr=0.0001, amsgrad=True), metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
es= EarlyStopping(monitor='val_loss', mode ='min', verbose = 0, patience = 20)
mc = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only = True, mode ='min', verbose = 0)

In [ ]:
model.fit_generator(generator=data_train,              
    steps_per_epoch=len(data_train),
    validation_data=data_test,                    
    validation_steps=len(data_test),
    epochs=50,
    callbacks = [es, mc], 
    use_multiprocessing = True,
    verbose=1)

In [ ]:
from keras.models import load_model
model = load_model('model.h5')

In [ ]:
submission_df = pd.read_csv('../input/sample_submission.csv')
submission_df['id_code'] = submission_df['id_code'].astype(str)+'.png'

In [ ]:
submission_datagen=keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
submission_gen=submission_datagen.flow_from_dataframe(
    dataframe=submission_df,
    directory = '../input/test_images',
    target_size=(192, 256),
    color_mode='rgb',
    classes=None,
    class_mode=None,
    batch_size=32,
    shuffle=False,
    seed=None,
    save_to_dir=None,
    save_prefix='',
    save_format='png',
    follow_links=False,
    x_col="id_code",
    interpolation='nearest')

In [ ]:
predictions=model.predict_generator(submission_gen, steps = len(submission_gen))

In [ ]:
max_probability = np.argmax(predictions,axis=1) 

In [ ]:
submission_df.head()

In [ ]:
submission_df.describe()

In [ ]:
submission_df['id_code'] = submission_df['id_code'].str.split('.', n = 1, expand = True)[0]

In [ ]:
submission_df.head()

In [ ]:
submission_df['diagnosis'] = max_probability
submission_df.to_csv('submission.csv', index=False)